# Bookmyshow tickets alert

## Import packages

In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
import webbrowser

In [3]:
import datetime

In [4]:
import urllib
import re

In [5]:
import requests

In [6]:
from bs4 import BeautifulSoup

In [7]:
!pip uninstall jupyternotify -y
!pip install git+https://github.com/cphyc/jupyter-notify.git
%reload_ext jupyternotify

Found existing installation: jupyternotify 0.1.15
Uninstalling jupyternotify-0.1.15:
  Successfully uninstalled jupyternotify-0.1.15


  Running command git clone -q https://github.com/cphyc/jupyter-notify.git 'C:\Users\tarun.yellapu\AppData\Local\Temp\pip-req-build-pasbq7rt'


  Cloning https://github.com/cphyc/jupyter-notify.git to c:\users\tarun.yellapu\appdata\local\temp\pip-req-build-pasbq7rt
  Created wheel for jupyternotify: filename=jupyternotify-0.1.15-py3-none-any.whl size=9299 sha256=032aaeb11550d5a884ff2accccb67f9549ef0302fb3f25f552930ef60de9e98f
  Stored in directory: C:\Users\tarun.yellapu\AppData\Local\Temp\pip-ephem-wheel-cache-urchl3vx\wheels\81\fa\76\14c6c94dd4279f0e0dd80914e64590761e951de08adb9842c3
Successfully built jupyternotify


<IPython.core.display.Javascript object>

## Config

In [10]:
# Bookmyshow URL
bms_url = 'https://in.bookmyshow.com/buytickets/sarkaru-vaari-paata-vijayawada/movie-vija-ET00131962-MT/20220513'
latest_date = 13

In [11]:
svp_title_song = "https://youtu.be/EqyKcZibh-Y?t=46"

## Find when bookings are open for 12th

In [11]:
%%notify -o -m "Book your tickets"
iterations = 1
while latest_date>13:
    
    # Ping website
    response = requests.request('get', url = bms_url,
                               headers={'Cache-Control': 'no-cache'})
    #response.text
    if response.status_code != 200:
        print("FATAL ERROR")
    bms_soup = BeautifulSoup(response.text)

    dates = bms_soup.find_all(name="a",
                  recursive=True,
                  class_ = "date-href")

    a = dates[0]
    latest_date = int(re.findall(pattern = r'\d+', string = a.div.text)[0])
    iterations = iterations+1
    #break
    if iterations%60==0:
        print(f"Ran for {int(iterations/60)} hour")
        #break
    time.sleep(60)
    
    #break
print(f"Tickets for SVP available from {latest_date} May")
time.sleep(2)
webbrowser.open_new(svp_title_song)

Tickets for SVP available from 13 May


True

<IPython.core.display.Javascript object>

## Notify when there is a new theatre

In [ ]:
%%notify -o
#-m "Book your tickets"
latest_date = 13
old_theatres = []
iterations = 1
new_available_theatre = []
old_available_theatre = 0
while latest_date==13:
    response = requests.request('get', url = bms_url,
                                   headers={'Cache-Control': 'no-cache'})
    #response.text
    if response.status_code != 200:
        print("FATAL ERROR")
    bms_soup = BeautifulSoup(response.text)

    
    
    
    if iterations%60==0:
        print(f"Ran for {int(iterations/60)} hour")
        #break
    

    theatres = bms_soup.find_all(name="a",
                                  recursive=True,
                                  class_ = "__venue-name")
    theatres = [theatre.text.strip() for theatre in theatres]
    new_theatres = list(set.difference(set(theatres),
                                      set(old_theatres)))
    if len(new_theatres)>0:
        print("Added new theatres")
        print(datetime.datetime.now())
        new_theatres.sort()
        for new_theatre in new_theatres:
            old_theatres.append(new_theatre)
            print(f"\n{new_theatre}")
            shows = bms_soup.find_all(name = "li",
                              attrs = {"class": "list",
                                       "data-name":new_theatre})[0]
            shows = shows.find_all(name = "a",
                                      attrs={"class":"showtime-pill",
                                       "data-availability":"A"})
            if len(shows)==0:
                print("$$$$ Haalu Nindindi $$$$")
            for show in shows:
                rem_seats = int(show["data-seats-percent"])
                show_time = show.text.strip()
                print(f"\t{show_time}: {rem_seats}%")
                
            
        time.sleep(8)
        #play song
        webbrowser.open_new(svp_title_song)
        print("\n")
    else:
        for theatre in set.difference(set(theatres),
                                      new_available_theatre):
            counter = 0
            shows = bms_soup.find_all(name = "li",
                              attrs = {"class": "list",
                                       "data-name":theatre})[0]

            for show in shows.find_all(name = "a",
                                      attrs={"class":"showtime-pill",
                                       "data-availability":"A"}):
                
                rem_seats = int(show["data-seats-percent"])
                show_time = show.text.strip()
                
                if rem_seats>40:
                    counter+=1
                    print(f"{theatre} {show_time}: {rem_seats}%")
                    #break
                    
            if counter>1 & (theatre not in new_available_theatre):
                new_available_theatre.append(theatre)
                #time.sleep(10)
                #play song
                webbrowser.open_new(svp_title_song)
                #break
        #break          
    #break  
    time.sleep(30)
        
    iterations = iterations+1

Added new theatres
2022-05-13 21:36:05.940624

Balaji Iconia A/c 4K Dolby Surround: Ibrahimpatnam
	09:30 PM: 20%

Cinepolis: PVP Square Mall, Vijayawada
	09:50 PM: 5%
	10:20 PM: 13%
	10:50 PM: 15%

Cinepolis: Power One Mall, Vijayawada
	10:20 PM: 8%
	10:50 PM: 14%

G3 Raj Yuvraj (4K Dolby Atmos): Vijayawada
$$$$ Haalu Nindindi $$$$

INOX: LEPL Icon, Patamata
	10:20 PM: 6%
	10:50 PM: 10%

INOX: Urvasi Complex, Gandhi Nagar
	10:00 PM: 14%
	11:00 PM: 30%

Miraj Cinemas, Shanti Prashanti: Vijayawada
	11:00 PM: 40%

PVR: Ripples, Vijaywada
	10:40 PM: 22%
	11:00 PM: 22%

Ravindra Cinemas: Vijayawada
$$$$ Haalu Nindindi $$$$

Sairam Screens A/C Dolby 7.1 Surround: Vijayawada
$$$$ Haalu Nindindi $$$$

Sivakrishna Cinemas A/C 2K DTS: Kondapalli
	09:30 PM: 18%

Swarna Cinemax: Ibrahimpatnam
	09:30 PM: 35%

Swarna Multiplex: Vijayawada
	10:00 PM: 18%

Y Screens: PNBS, Vijayawada
	10:30 PM: 29%
	11:00 PM: 100%


Y Screens: PNBS, Vijayawada 11:00 PM: 90%
Y Screens: PNBS, Vijayawada 11:00 PM: 90%
Y 

In [12]:
shows

<li class="list" data-allow-sales="Y" data-has-cod="false" data-has-cop="false" data-has-mticket="false" data-has-unpaid="false" data-id="JTSC" data-is-down="false" data-is-food-sales="N" data-is-full-seat-layout="Y" data-is-multiplex="Y" data-is-new-cinema="N" data-lat="16.5132" data-lng="80.6263" data-message="1. Only Online booking partner server messages are allowed. Forwarded messages are not allowed.&lt;br&gt;2. A ticket is mandatory for 3 years and above.&lt;br&gt;3. Entry is allowed only for valid ticket holders.&lt;br&gt;4. In case the ticket is lost or misplaced, a duplicate ticket can`t be issued.&lt;br&gt;5. Tickets once purchased can`t be canceled, exchanged or refunded.&lt;br&gt;6. Outside food and beverages are not allowed in the cinema hall.&lt;br&gt;7. Decisions taken by Swarna Multiplex shall be final and binding.&lt;br&gt;8. People who have consumed alcohol will not be allowed into the cinema hall.&lt;br/&gt;9. Please note that customers will have to pay Extra Charge